#Trabajo Practico POO
##Sistema Farmacia

###Paquete Filtros

In [0]:
public interface Filtro {
    public abstract boolean cumple(Droga d);
}

In [0]:
public class FDroga implements Filtro {

    private Droga droga;

    public FDroga(Droga droga){
        this.droga = droga;
    }

    public boolean cumple(Droga d){
        return d.contiene(d);
    }
}

In [0]:
public class FContraindic implements Filtro {

    private Sintoma contraindic;

    public FContraindic(Sintoma contraindic){
        this.contraindic = contraindic;
    }

    public boolean cumple(Droga d){
        return (d != null) && d.esContraindic(contraindic);
    }
}

In [0]:
public class FAccTerap implements Filtro {
	
	private EstPato accion;
	
	public FAccTerap(EstPato accion) {
		this.accion = accion;
	}
	
    public boolean cumple(Droga d){
        return (d != null) && d.esAccionTerap(this.accion);
    }
    
}

In [0]:
public class FAnd implements Filtro {

    private ArrayList<Filtro> filtros = new ArrayList<Filtro>();

    public FAnd(Filtro f1, Filtro f2){
        this.filtros.add(f1);
        this.filtros.add(f2);
    }

    public FAnd(ArrayList<Filtro> filtros) {
        this.filtros = filtros;
    }

    public boolean cumple(Droga d) {
        for(Filtro f : filtros){
            if(!f.cumple(d))
                return false;
        }
        return true;
    }

}

In [0]:
public class FOr implements Filtro{

    private ArrayList<Filtro> filtros;

    public FOr(ArrayList<Filtro> filtros) {
        this.filtros = filtros;
    }

    public boolean cumple(Droga d) {

        for(Filtro f : filtros){
            if(f.cumple(d))
                return true;
        }
        return false;
    }
}


In [0]:
public class FNot implements Filtro{
    Filtro f;
    public FNot(Filtro f){
        this.f = f;
    }
    public boolean cumple(Droga d){
        return !f.cumple(d);
    }
}

###Paquete Drogas

In [0]:
public abstract class Droga {

    protected String nombre = "Unnamed";

    public String getNombre(){
        return nombre;
    }
    public abstract boolean contiene(Droga d);
    public abstract float getPorcentDroga(Droga d);
    public abstract boolean esAccionTerap(EstPato estPato);
    public abstract boolean esContraindic(Sintoma sintoma);
    public abstract ArrayList<Droga> getDrogas(Filtro filtro);
    public abstract ArrayList<Medicamento> getMeds(Filtro filtro);

}

In [0]:
public class DrogaSimple extends Droga {

    private ArrayList<EstPato> accionesTerap;
    private ArrayList<Sintoma> contraindics;

    public DrogaSimple(String nombre, ArrayList<EstPato> accionesTerap, ArrayList<Sintoma> contraindics) {
        this.nombre = nombre;
        this.accionesTerap = accionesTerap;
        this.contraindics = contraindics;
    }

    public boolean contiene(Droga d){
        return nombre.equals(d.getNombre());
    }

    public float getPorcentDroga(Droga d){
        return (d.getNombre() == this.nombre)? (float)1:(float)0;
    }

    public boolean esAccionTerap(EstPato estPato){
        return accionesTerap.contains(estPato);
    }

    public boolean esContraindic(Sintoma sintoma){
        return contraindics.contains(sintoma);
    }

    public ArrayList<Droga> getDrogas(Filtro filtro){
        ArrayList<Droga> res = new ArrayList<Droga>();
        
        if (filtro.cumple(this))
        	res.add(this);
        
        return res;
    }

    public ArrayList<Medicamento> getMeds(Filtro filtro){
        ArrayList<Medicamento> res = null;

        if(filtro.cumple(this)) res = new ArrayList<Medicamento>();

        return res;
    }
}

In [0]:
public class Medicamento extends Droga {

    private ArrayList<Droga> compuestos;
    private float[] concentracion;

    public Medicamento(String nombre, ArrayList<Droga> compuestos, float[] concentracion){
        this.nombre = nombre;
        this.compuestos = new ArrayList<Droga>();
        this.compuestos.addAll(compuestos);
        int tam = compuestos.size();
        this.concentracion = new float[tam];

        for(tam -= 1; tam > -1 ; tam--){
            this.concentracion[tam] = concentracion[tam];
        }

    }

    public boolean contiene(Droga d){
        for(Droga c : compuestos){
            if(c.contiene(d))
                return true;
        }
        return false;
    }

    //Devuelve el porcentaje de DrogaSimple o Medicamento que contiene this
    public float getPorcentDroga(Droga d){
        float res = (float)0.0;
        if(d.getNombre() != this.nombre)
            for(int i = 0 ; i < compuestos.size() ; i++){
                res += concentracion[i]*compuestos.get(i).getPorcentDroga(d);
            }
        else
            return (float)1;

        return res;
    }

    public boolean esAccionTerap(EstPato estPato){
        for(Droga d : compuestos){
            if(d.esAccionTerap(estPato))
                return true;
        }
        return false;
    }

    public boolean esContraindic(Sintoma sintoma){
        for(Droga d : compuestos){
            if(d.esContraindic(sintoma))
                return true;
        }
        return false;
    }

    public ArrayList<Droga> getDrogas(Filtro filtro){
        ArrayList<Droga> res = new ArrayList<Droga>();

        boolean cumple = false;

        for(Droga d : compuestos){
            ArrayList<Droga> aux = d.getDrogas(filtro);
            if(aux.size() > 0){
                for(Droga dAux : aux){
                    if (!res.contains(dAux))
                        res.add(dAux);
                }
                cumple = true;
            }
        }

        if(cumple)
            res.add(this);
        
        return res;
    }

    public ArrayList<Medicamento> getMeds(Filtro filtro){

        ArrayList<Medicamento> res = new ArrayList<Medicamento>();
        boolean cumple = false;

        for(Droga d : compuestos){
            ArrayList<Medicamento> aux = d.getMeds(filtro);

            if(aux != null){///solo devuelve null si era DrogaSimple que no cumple filtro
                if(aux.size() > 0)
                    for(Medicamento dAux : aux){
                        if (!res.contains(dAux))
                            res.add(dAux);
                    }

                cumple = true;
            }
        }

        if(cumple)
            res.add(this);

        return res;

    }
}

###Paquete MainPack

In [0]:
public enum EstPato {
    Cefalea,
    Gripe,
    Congestion,
    E1,
    E2,
    E3,
    E4
}

In [0]:
public enum Sintoma {
    Embarazo,
    Lactancia,
    Cardiaco,
    S1,
    S2,
    S3,
    S4
}

In [0]:
public class Enfermedad {
    private String nombre;
    private ArrayList<EstPato> ePatos;

    public Enfermedad(String nombre, ArrayList<EstPato> ePatos) {
        this.nombre = nombre;
        this.ePatos = ePatos;
    }

    public String getNombre() {
        return nombre;
    }

    public ArrayList<EstPato> getEPatos() {
        return ePatos;
    }

    public String toString(){
        String salida = "Nombre de Enfermedad: " + nombre + "\n";
        salida += "Estados Patologicos que Trata: \n";
        for(EstPato e : ePatos)
            salida += e;

        return salida;
    }
}

####Clase Paciente

Se considera a un paciente como un contenedor de Sintomas y Alergias, las cuales solo son necesarias para filtrar los medicamentos que se le pueden o no otorgar para una enfermedad que este contraiga.

In [0]:
public class Paciente {
    private ArrayList<DrogaSimple> alergias = new ArrayList<DrogaSimple>();
    private ArrayList<Sintoma> sintomas = new ArrayList<Sintoma>();

    public Paciente(Sintoma s){
        this.sintomas.add(s);
    }

    public Paciente(DrogaSimple alergia, Sintoma sintoma) {///PARA PRUEBAAAASSSS
        this.alergias.add(alergia);
        this.sintomas.add(sintoma);
    }

    public Paciente(ArrayList<DrogaSimple> alergias, ArrayList<Sintoma> sintomas) {
        this.alergias = alergias;
        this.sintomas = sintomas;
    }

    public ArrayList<DrogaSimple> getAlergias(){
    	return this.alergias;
    }
    
    public ArrayList<Sintoma> getSintomas(){
    	return this.sintomas;
    }

    public String toString(){
        String salida = "Alergias del Paciente: \n(";
        for(DrogaSimple e : alergias)
            salida += e.getNombre() + ", ";

        salida += ")\nSintomas del Paciente: \n(";
        for(Sintoma e : sintomas)
            salida += e + ", ";

        return salida + ")";
    }
    
}

####Clase Farmacia
Es la clase que contiene el metodo main, en la misma se proveen los servicios solicitados por el enunciado y sus respectivos test. 

Dado que es un sistema de control de medicamentos, solo se mantienen estructuras para estos, sin considerar las drogas simples. Ademas, se mantiene cuenta del stock en un arreglo que se corresponde con los medicamentos.

Siendo que esta farmacia puede tener clientes y estos podrian frecuentarla, se mantienen los datos de Sintomas y Alergias de ellos en un arreglo tambien.

In [0]:
public class Farmacia {

    private static ArrayList<Medicamento> medicamentos = new ArrayList<Medicamento>();//Suponemos que al ser una farmacia no vende drogas simples, solo medicamentos.
    private static ArrayList<Integer> stock = new ArrayList<>();
    private static ArrayList<Paciente> pacientes = new ArrayList<Paciente>();

    public static void main(String[] args){
        inicArrs();
        //testFiltradoDroga();//OK
        //testRecetarAEnf();//OK
        //testRecetarAEnfPac();//OK
        //testRecetarDrogas();//ok
        //testPorcentajes();//ok


    }

    public static ArrayList<Medicamento> recetarMed(Enfermedad enf){
    	ArrayList<Medicamento> res = new ArrayList<Medicamento>();
    	
    	ArrayList<EstPato> estadosPatologicos = enf.getEPatos();

    	for(Medicamento m: medicamentos){
    	    boolean cumple = true;
            for(EstPato e: estadosPatologicos)
                if (!m.esAccionTerap(e)){
                    cumple = false;
                    break;
                }

            if(cumple && (stock.get(medicamentos.indexOf(m)) > 0))
                res.add(m);
        }

    	return res;
    	
    }

    public static ArrayList<Droga> recetarDrog(Enfermedad enf){
        ArrayList<Droga> res = new ArrayList<Droga>();

        ArrayList<EstPato> estadosPatologicos = enf.getEPatos();
        ArrayList<Filtro> filtros = new ArrayList<Filtro>();
        for(EstPato e : estadosPatologicos)
            filtros.add(new FAccTerap(e));

        Filtro f = new FAnd(filtros);

        for(Medicamento m: medicamentos){
            ArrayList<Droga> aux = m.getDrogas(f);
            for(Droga d : aux )
                if(!res.contains(d))
                    res.add(d);
        }


        return res;

    }
    
    /*TODO: ESPERAR RESPUESTA DEL PROFE
            *consultar por los constructores con ArrayList de los FLogicos
            *getMeds devuelve los medicamentos por los que esta compuesto M
            *getDrogas devuelve Medicamento y DrogaSimple del que esta hecho
            *para recetar, que pasa si M2 es la cura y no lo tengo, M1 tiene M2 => solo deveria devolver M1
    */
    public static ArrayList<Medicamento> recetarMed(Enfermedad enf, Paciente pac) {
        ArrayList<Medicamento> res = new ArrayList<Medicamento>();
        FAnd f;
        {
            ArrayList<DrogaSimple> alergias = pac.getAlergias();
            ArrayList<Sintoma> sintomas = pac.getSintomas();
            ArrayList<EstPato> ePatos = enf.getEPatos();

            ArrayList<Filtro> fsAlergias = new ArrayList<Filtro>();//drogas que no puede consumir
            ArrayList<Filtro> fsSintomas = new ArrayList<Filtro>();//contraindicaciones que no puede tener
            ArrayList<Filtro> fsAccTerap = new ArrayList<Filtro>();//ePatologicos que debe curar

            for (DrogaSimple d : alergias)
                fsAlergias.add(new FDroga(d));
            for (Sintoma s : sintomas)
                fsSintomas.add(new FContraindic(s));
            for (EstPato e : ePatos)
                fsAccTerap.add(new FAccTerap(e));

            ArrayList<Filtro> filtros = new ArrayList<Filtro>();
            filtros.add(new FAnd(fsAccTerap));
            filtros.add(new FNot(new FOr(fsAlergias)));
            filtros.add(new FNot(new FOr(fsSintomas)));

            f = new FAnd(filtros);
        }

        for (Medicamento m : medicamentos)
            if(f.cumple(m) && (stock.get(medicamentos.indexOf(m)) > 0))
                res.add(m);

        return res;
    }
    
    public static float getPorcentajeMed(Medicamento m, Droga d) {
    	return m.getPorcentDroga(d);
    }

    public static ArrayList<Medicamento> getMedsPorcentaje(Droga d, float porcent){
        ArrayList<Medicamento> res = new ArrayList<Medicamento>();
        for(Medicamento m : medicamentos){
            if(m.getPorcentDroga(d) > porcent)
                res.add(m);
        }
        return res;
    }

    private static void inicArrs(){

        //INICIALIZANDO DROGAS SIMPLES
        ArrayList<EstPato> accTerap5 = new ArrayList<EstPato>();
        ArrayList<Sintoma> contra5 = new ArrayList<Sintoma>();
        accTerap5.add(EstPato.E2);
        contra5.add(Sintoma.Lactancia);
        DrogaSimple d5 = new DrogaSimple("d5", accTerap5, contra5);

        ArrayList<EstPato> accTerap0 = new ArrayList<EstPato>();
        ArrayList<Sintoma> contra0 = new ArrayList<Sintoma>();
        accTerap0.add(EstPato.E1);
        contra0.add(Sintoma.S1);
        DrogaSimple d0 = new DrogaSimple("d0", accTerap0, contra0);

        ArrayList<EstPato> accTerap1 = new ArrayList<EstPato>();
        ArrayList<Sintoma> contra1 = new ArrayList<Sintoma>();
        accTerap1.add(EstPato.E2);
        contra1.add(Sintoma.S2);
        DrogaSimple d1 = new DrogaSimple("d1", accTerap1, contra1);

        ArrayList<EstPato> accTerap2 = new ArrayList<EstPato>();
        ArrayList<Sintoma> contra2 = new ArrayList<Sintoma>();
        accTerap2.add(EstPato.E3);
        contra2.add(Sintoma.S3);
        DrogaSimple d2 = new DrogaSimple("d2", accTerap2, contra2);

        ArrayList<EstPato> accTerap3 = new ArrayList<EstPato>();
        ArrayList<Sintoma> contra3 = new ArrayList<Sintoma>();
        accTerap3.add(EstPato.E4);
        contra3.add(Sintoma.S4);
        DrogaSimple d3 = new DrogaSimple("d3", accTerap3, contra3);

        ArrayList<EstPato> accTerap4 = new ArrayList<EstPato>();
        ArrayList<Sintoma> contra4 = new ArrayList<Sintoma>();
        accTerap4.add(EstPato.Gripe);
        contra4.add(Sintoma.Cardiaco);
        DrogaSimple d4 = new DrogaSimple("d4", accTerap4, contra4);

        //INICIALIZANDO MEDICAMENTOS
        float[] cons = {(float) 0.5, (float) 0.5};

        ArrayList<Droga> comp5 = new ArrayList<Droga>();
        comp5.add(d1);
        comp5.add(d0);
        Medicamento m5 = new Medicamento("m5", comp5, cons);

        ArrayList<Droga> comp4 = new ArrayList<Droga>();
        comp4.add(d3);
        comp4.add(d4);
        Medicamento m4 = new Medicamento("m4", comp4, cons);

        ArrayList<Droga> comp3 = new ArrayList<Droga>();
        comp3.add(d2);
        comp3.add(m4);
        Medicamento m3 = new Medicamento("m3", comp3, cons);

        ArrayList<Droga> comp2 = new ArrayList<Droga>();
        comp2.add(m3);
        comp2.add(m5);
        Medicamento m2 = new Medicamento("m2", comp2, cons);

        ArrayList<Droga> comp1 = new ArrayList<Droga>();
        comp1.add(m2);
        comp1.add(d5);
        comp1.add(d1);
        float[] cons1 = {(float) 0.5, (float) 0.5, (float) 0.5};
        Medicamento m1 = new Medicamento("m1", comp1, cons1);

        ///CARGO ESTRUCTURAS DEL SISTEMA
        medicamentos.add(m1);
        stock.add(5);
        medicamentos.add(m2);
        stock.add(5);
        medicamentos.add(m3);
        stock.add(5);
        medicamentos.add(m4);
        stock.add(5);
        medicamentos.add(m5);
        stock.add(0);


        pacientes.add(new Paciente(d1, Sintoma.Cardiaco));
        pacientes.add(new Paciente(d3, Sintoma.S2));

    }

    static void testFiltradoDroga(){
        System.out.println("Inicio test filtrado por Droga");
        System.out.println("Resultado esperado: M3, M4");
        
        Medicamento m = medicamentos.get(2);
        DrogaSimple d4 = new DrogaSimple("d4", new ArrayList<EstPato>(), new ArrayList<Sintoma>());
        ArrayList<Medicamento> medsFiltrados = m.getMeds(new FDroga(d4));
        
        System.out.println("Tamaño resultado: " + medsFiltrados.size());
        if(medsFiltrados.size() > 0)
            for(Medicamento med : medsFiltrados)
                System.out.println(med.getNombre());

        System.out.println("FIN test filtrado por Droga");
    }
    

    static void testRecetarAEnf(){
        System.out.println("Inicio test recetar Medicamento a Enfermedad");
        System.out.println("Resultado esperado: M1, M2"); //M5 seria parte si tuviera stock
        ArrayList<EstPato> estados = new ArrayList<EstPato>();
        estados.add(EstPato.E1);
        //estados.add(EstPato.E2);

        Enfermedad e = new Enfermedad("Lepra", estados);
        System.out.println("Enfermedad a recetar:");
        System.out.println(e);

        ArrayList<Medicamento> receta = recetarMed(e);

        System.out.println("RECETA: ");
        for (Medicamento m : receta)
            System.out.println(m.getNombre());

        System.out.println("FIN test recetar Medicamento a Enfermedad");
    }

    static void testRecetarAEnfPac(){
        System.out.println("Inicio test recetar Medicamento a Paciente de Riesgo");
        System.out.println("Resultado esperado: M2");//M5 seria parte si tuviera stock

        ArrayList<EstPato> estados = new ArrayList<EstPato>();
        estados.add(EstPato.E2);
        Enfermedad e = new Enfermedad("Alzheimer", estados);

        Sintoma sintoma = Sintoma.Lactancia;
        Paciente paciente = new Paciente(sintoma);
        System.out.println(paciente);

        System.out.println("Enfermedad a recetar:");
        System.out.println(e);

        ArrayList<Medicamento> receta = recetarMed(e,paciente);

        System.out.println("RECETA: ");
        for (Medicamento m : receta)
            System.out.println(m.getNombre());

        System.out.println("FIN test recetar Medicamento a Paciente de Riesgo");
    }

    static void testRecetarDrogas(){
        System.out.println("Inicio test recetar Drogas a Enfermedad");
        System.out.println("Resultado esperado: M1, M2, M5, D0");
        ArrayList<EstPato> estados = new ArrayList<EstPato>();
        estados.add(EstPato.E1);
        //estados.add(EstPato.E2);

        Enfermedad e = new Enfermedad("Lepra", estados);
        System.out.println("Enfermedad a recetar:");
        System.out.println(e);

        ArrayList<Droga> receta = recetarDrog(e);

        System.out.println("RECETA: ");
        for (Droga m : receta)
            System.out.println(m.getNombre());

        System.out.println("FIN test recetar Drogas a Enfermedad");
    }

    static void testPorcentajes(){
        System.out.println("Inicio test Porcentaje de droga en un medicamento");
        ArrayList<EstPato> accTerap1 = new ArrayList<EstPato>();
        ArrayList<Sintoma> contra1 = new ArrayList<Sintoma>();
        accTerap1.add(EstPato.E2);
        contra1.add(Sintoma.S2);
        DrogaSimple d1 = new DrogaSimple("d1", accTerap1, contra1);

        Medicamento med = medicamentos.get(1);

        System.out.println("Medicamento: " + med.getNombre());
        System.out.println("Droga: " + d1.getNombre());
        System.out.println("Resultado esperado= 0,25");



        float res= getPorcentajeMed(med, d1);

        System.out.println("Resultado ="+ res);
        System.out.println("FIN test Porcentaje de droga en un medicamento");
/////////////////////////////////////////////////////////////////////////////////////////////////////////////
        System.out.println("Inicio test medicamentos con porcentaje de droga mayor al dado");
        float porcent = (float) 0.4;

        System.out.println("Porcentaje Dado: " + porcent);
        System.out.println("Droga: " + d1.getNombre());
        System.out.println("Resultado esperado= m1,m5");



        ArrayList<Medicamento> res1 = getMedsPorcentaje(d1, porcent);

        System.out.println("Resultado =");
        for(Medicamento m : res1){
            System.out.println(m.getNombre());
        }
        System.out.println("FIN test medicamentos con porcentaje de droga mayor al dado");
    }
}